In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
import calendar



In [2]:
def date_time_index(df):
    df['datetime'] = pd.to_datetime(df['Datum']+df['Zeit'],format= "%d-%m-%y%H:%M:%S")
    df.drop(['Datum','Zeit'],inplace=True,axis=1)
    df['Wert'] = df['Wert'].str.replace(',', '.')
    df=df.set_index('datetime')
    return df

In [8]:
def merge_solar(lis,name):
    
    power = pd.read_csv('{}.CSV'.format(lis[0]),sep=';',header = 4)
    power.drop('Status',inplace=True,axis=1)
    power_df = date_time_index(power)
    
    
    global_radiation = pd.read_csv('{}.CSV'.format(lis[1]),sep=';',header = 4)
    global_radiation.drop('Status',inplace=True,axis=1)
    global_radiation_df = date_time_index(global_radiation)
    
    
    
    temperature = pd.read_csv('{}.CSV'.format(lis[2]),sep=';',header = 4)
    temperature.drop('Status',inplace=True,axis=1)
    temperature_df = date_time_index(temperature)
    temperature_df = temperature_df.resample('15T').sum().transform(lambda x: x.replace(to_replace=0, method='ffill'))

    merge_df = power_df.merge(global_radiation_df,left_index=True, right_index=True)
    merge_df = merge_df.merge(temperature_df,left_index=True, right_index=True)

    merge_df.columns = ['power','global_radiation','temperature']
    
    merge_df.to_csv('{}.csv'.format(name),sep=',')

In [17]:
merge_solar(['EWSKE_SUPW','KWM4ANK1PW','KLTEMP05PW'],'Kenn')
merge_solar(['EWSKMESUPW','KWM4ANK1PW','KLTEMP05PW'],'MehringIII')
merge_solar(['KWM4ANE3PW','KWM4ANK1PW','KLTEMP05PW'],'Mehring IV')